In [12]:
import pandas as pd
import numpy as np

Lectura de dades

In [13]:
activitats_v = pd.read_csv("data/activitats_valencia.csv")
activitats_a = pd.read_csv("data/activitats_alacant.csv")
activitats_c = pd.read_csv("data/activitats_castello.csv")
ajuntaments_info = pd.read_csv("data/ajuntaments_info.csv")

* Seleccio de variables i canvi de nom

In [4]:
columns = ["Marca temporal", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
columns_v = ["Marca de temps", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
column_rename = ["marca_temps", "localitat", "data", "categoria", "categoria_alternativa", "nom_activitat", "companyia", "lloc", "llengua_activitat", "preu", "regidoria", "llengua_publicitat", "font", "persona_dades"]

In [14]:
activitats_v = activitats_v[columns_v]
activitats_v.columns = column_rename
activitats_c = activitats_c[columns]
activitats_c.columns = column_rename
activitats_a = activitats_a[columns]
activitats_a.columns = column_rename

* Juntem els 3 datasets

In [15]:
activitats = pd.concat([activitats_a, activitats_v, activitats_c], axis = 0)

In [16]:
activitats.shape

(6950, 14)

* Preprocessament de dades

* Creació de noves variables

Preu_mitjà

In [17]:
activitats["preu_mitja"] = activitats.preu.str.findall(r"\d+[,'.]?\d*")
activitats["preu_mitja"].loc[activitats.preu.notnull()] = activitats.loc[activitats.preu.notnull()].preu_mitja.apply(lambda x: np.mean([float(num.replace("'", ".").replace(",", ".")) for num in x]))

c:\Users\quimm\Anaconda3\envs\venv\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [19]:
activitats.preu_mitja.loc[activitats.preu_mitja.notnull()]

65      5.0
93     40.0
140     7.0
142     3.0
144     7.0
       ... 
122     3.0
123     3.0
124     3.0
125     3.0
126     3.0
Name: preu_mitja, Length: 1173, dtype: object

Categoria New

In [20]:
def assign_category(category_alternative):

    if type(category_alternative) != str:
        category_new = np.nan
    else:
        category_alternative = category_alternative.lower()
        if "fall" in category_alternative:
            category_new = "Falles"
        elif "màg" in category_alternative:
            category_new = "Màgia"
        elif "visita" in category_alternative:
            category_new = "Activitats sobre patrimoni"
        elif "circ" in category_alternative:
            category_new = "Circ"
        elif "concu" in category_alternative:
            category_new = "Concurs"
        elif "premi" in category_alternative:
            category_new = "Premis"
        elif "taller" in category_alternative:
            category_new = "Taller"
        elif "espectac" in category_alternative:
            category_new = "Espectacle"
        else:
            category_new = np.nan

    return category_new


In [21]:
activitats["categoria_new"] = activitats["categoria_alternativa"].apply(assign_category)
activitats["categoria_new"] = np.where(activitats['categoria_new'].isnull(), activitats['categoria'], activitats['categoria_new'])

* Mes

In [23]:
activitats.data

0       1/10/2022
1       1/10/2022
2       1/10/2022
3       1/10/2022
4       1/10/2022
          ...    
951    27/03/2023
952    24/03/2023
953    27/03/2023
954     7/03/2023
955    23/03/2023
Name: data, Length: 6950, dtype: object

In [24]:
pd.to_datetime(activitats.data, format= "%d/%m/%Y")

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 223-02-10 00:00:00

In [10]:
activitats["Mes"]

str

* Agrupacions

llengua activitat

In [153]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(3).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_activitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top2 = grup1.groupby("localitat").apply(lambda x : x.head(2).tail(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top3 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

llengua publicitat

In [150]:
grup1 = activitats.groupby(["localitat","llengua_publicitat"])["llengua_publicitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(2).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_publicitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_publicitat_top2 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

Language position

In [297]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

In [304]:
pos_val = grup1.groupby("localitat").apply(lambda x : x.index[x["llengua_activitat"]=="Valencià/català"])
pos_val = pd.DataFrame(pos_val)

pos_cas = grup1.groupby("localitat").apply(lambda x : x.index[x["llengua_activitat"]=="Espanyol"])
pos_cas = pd.DataFrame(pos_cas)

In [291]:
def get_position(array):
    try:
        return array[0][0][1]
    except:
        return np.nan

In [301]:
positions_val = []

for i in range(pos_val.shape[0]):
    positions_val.append(get_position(pos_val.iloc[i]))

In [305]:
positions_cas = []

for i in range(pos_cas.shape[0]):
    positions_cas.append(get_position(pos_cas.iloc[i]))

* Creating datasets

activitats_summaries

In [233]:
activitats_summaries = pd.DataFrame(llengua_activitat_top1)
activitats_summaries = activitats_summaries.reset_index()
del activitats_summaries["level_1"]
activitats_summaries.columns = ['localitat', 'llengua_activitat_top1']

In [313]:
activitats_summaries['llengua_activitat_top2'] = llengua_activitat_top2.values
activitats_summaries['llengua_activitat_top3'] = llengua_activitat_top3.values
activitats_summaries['llengua_publicitat_top1'] = llengua_publicitat_top1.values
activitats_summaries['llengua_publicitat_top2'] = llengua_publicitat_top2.values
activitats_summaries["position_val_in_llengua_activitat"] = positions_val
activitats_summaries["position_esp_in_llengua_activitat"] = positions_cas

In [310]:
activitats_summaries

,localitat,llengua_activitat_top1,llengua_activitat_top2,llengua_activitat_top3,llengua_publicitat_top1,llengua_publicitat_top2,position_val_in_llengua_publicitat,position_esp_in_llengua_publicitat
0,Alacant,Espanyol,Activitat sense llengua,No hi ha informació,Espanyol,Bilingüe,3.0,0.0
1,Alaquàs,Espanyol,Activitat sense llengua,Valencià/català,Bilingüe,Valencià/català,2.0,0.0
2,Albal,Activitat sense llengua,Valencià/català,Espanyol,Valencià/català,Espanyol,1.0,2.0
3,Albatera,Espanyol,Activitat sense llengua,Activitat sense llengua,Espanyol,Espanyol,NaN,0.0
4,Alberic,Valencià/català,Activitat sense llengua,Espanyol,Valencià/català,Valencià/català,0.0,2.0
...,...,...,...,...,...,...,...,...
132,Vinaròs,Activitat sense llengua,Valencià/català,Espanyol,Valencià/català,Bilingüe,1.0,2.0
133,Xirivella,Activitat sense llengua,Espanyol,No hi ha informació,Bilingüe,Valencià/català,3.0,1.0
134,Xixona,Activitat sense llengua,Valencià/català,Espanyol,Valencià/català,Espanyol,1.0,2.0
135,Xàbia,Activitat sense llengua,Valencià/català,Espanyol,Bilingüe,Valencià/català,1.0,2.0


Saving Datasets

In [311]:
activitats.to_csv("data/activitats.csv")

In [314]:
activitats_summaries.to_csv("data/activitats_summaries.csv")